<center>
    <img src="https://github.com/juanandres-montero/macroeconometria/blob/main/logo%20EEC%20grande.png?raw=1" width="396" height="161"><br>
    <b style="font-size:200%">EC4301 MACROECONOMETRÍA</b><br>
    <b style="font-size:100%">Asistente: Juan Andrés Montero</b><br><br>
    <b style="font-size:100%"> Laboratorio #10:</b>
    <br>    
    <div style="font-size:150%; color:white; background-color: #0064b0; padding-top: 20px; padding-bottom: 20px; width: 50%; margin: 0 auto; border-radius: 15px; text-align: center;">
        Análisis de Cambio Estructural 2
    </div>
</center>

<br>

<i style="font-size:80%"> 
    Creado: Noviembre 2024
</i>

<hr>

## Resumen del laboratorio anterior

- El mercado de gasolina en Estados Unidos (1953-2004)
- La prueba de Chow para detectar cambio estructural
- El impacto de la crisis del petróleo de 1973
- La prueba de Andrews para varianzas desiguales

## Simulación de Series con Cambio Estructural
### 1. Proceso AR(1) con Cambio en el Nivel
Vamos a analizar un proceso autorregresivo con un cambio estructural en el nivel:

```stata
clear all
set obs 100
set seed 12345
gen t = _n
tsset t
* Generamos el proceso AR(1) con cambio estructural
gen y = 0
gen D = 0
replace D = 3 if t>50  // Cambio estructural en t=50
gen epsilon = rnormal(0, 1)
replace y = 0.5 * L.y + epsilon + D if t>1
* Visualización
tsline y
```

Este proceso sigue la forma:
y_{t} = 0.5y_{t-1} + ε_{t} + D_L 

donde:
- ε_{t} ~ N(0,1)
- D_L = 3 si t>50, 0 en otro caso
- El proceso es estacionario antes y después del cambio

### 2. Prueba de Raíz Unitaria con Cambio Estructural

La presencia de cambios estructurales puede llevar a conclusiones erróneas en las pruebas de raíz unitaria estándar. Veamos esto en práctica:

```stata
* Prueba Dickey-Fuller estándar
dfuller y
```
A pesar de que el proceso es estacionario, la prueba Dickey-Fuller puede fallar en rechazar la hipótesis nula de raíz unitaria debido al cambio estructural.

### 3. Prueba de Perron (1989)
Perron propuso una modificación a la prueba de raíz unitaria que considera explícitamente el cambio estructural:
Hipótesis nula (H0): y_{t} = α₀ + y_{t-1} + (μ₁)D_P + ε_t
Hipótesis alternativa (H1): y_{t} = α₀ + (α₂)t + (μ₂)D_L + ε_t

Donde:
- D_P = 1 si t = τ + 1 (momento del cambio)
- D_L = 1 si t > τ

```stata
* Generamos las variables dummy
gen D_P = 0
replace D_P = 1 if t == 51
gen D_L = 0
replace D_L = 1 if t>50

* Estimación del modelo combinado
regress y L.y t D_P D_L

predict residuos, resid
* Prueba de correlación serial
wntestq residuos

```

## Prueba CUSUM de Inestabilidad de Parámetros
La prueba CUSUM es útil para detectar cambios estructurales cuando no conocemos el momento exacto del cambio.

### 1. Simulación de Proceso con Cambio en Parámetros
```stata
clear all
set obs 200
set seed 2023
gen t = _n
tsset t

* Generamos proceso con cambio en parámetros
gen y = 0
gen epsilon = rnormal(0, 1)
replace y = 1 + 0.25*L.y + epsilon if t>1
replace y = 2.5 + 0.65*L.y + epsilon if t>=125

* Visualización
tsline y
```

Este proceso tiene:

- Cambio en la constante: 1 → 2.5
- Cambio en el coeficiente AR: 0.25 → 0.65
- Punto de cambio: t = 125

### 2. Implementación de la Prueba CUSUM
```stata
* Estimación AR(1)
regress y L.y

* Prueba CUSUM
estat sbcusum

* Para versiones anteriores de STATA
ssc install cusum9
cusum9 y L.y

```

## Interpretación y Conclusiones

1. Cambio Estructural y Raíz Unitaria:
   - Los cambios estructurales pueden confundirse con raíces unitarias
   - Es crucial considerar posibles cambios al analizar la estacionariedad

2. Prueba de Perron:
   - Permite distinguir entre raíz unitaria y cambio estructural
   - Es más poderosa cuando conocemos el punto de cambio

3. Análisis CUSUM:
   - Útil para detectar cambios cuando el punto es desconocido
   - Proporciona evidencia visual de la inestabilidad

# Referencias

- Perron, P. (1989). "The Great Crash, the Oil Price Shock, and the Unit Root Hypothesis." Econometrica, 57(6), 1361-1401.

- Brown, R.L., Durbin, J., and Evans, J.M. (1975). "Techniques for Testing the Constancy of Regression Relationships over Time." Journal of the Royal Statistical Society, Series B, 37(2), 149-192.